In [ ]:
import json
import itertools
import xgi
import numpy as np
import networkx as nx
from tqdm import tqdm
import pickle as pk

dataset='congress-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-committees_simplices';name_file='{0}'.format(dataset);
#dataset='house-committees_simplices';name_file='{0}'.format(dataset);
#dataset='email-Enron_simplices'; name_file='{0}'.format(dataset); 
#dataset='email-Eu_simplices'; name_file='{0}'.format(dataset); 
#dataset='hyperedges-cat-edge-algebra-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-geometry-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-music-blues-reviews'; name_file='{0}'.format(dataset);
#dataset='Mid1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Elem1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='InVS15'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='SFHH'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LH10'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LyonSchool'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Thiers13'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='M_PL_015_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_062_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);

with open('Data/{0}.json'.format(name_file)) as json_file:
    data = json.load(json_file)

data = [data[i] for i in range(len(data)) if len(data[i])>1] #select only interactions of size >=2
for i in range(len(data)):
    data[i].sort() #order the nodes according to their ID labels, e.g. [2,1] -> [1,2]
data.sort()
data = list(data for data,_ in itertools.groupby(data)) #delete duplicate hyperedges
data.sort(key = len)

L=len(data)
G = xgi.Hypergraph(data) #build hypergraph
N=len(G.nodes)

g=xgi.convert.convert_to_graph(G) #build the projected graph
idx_orig=list(g.nodes()) 
idx_prev=idx_orig

k_step=1
K=range(1,2000,k_step)

k_shell_dict={}

for j in idx_orig:
    k_shell_dict[j]=0

for y in tqdm(range(len(K))):
    kk = K[y]
    h = nx.k_core(g, kk) #identify nodes in k-core
    idx_removed = list(sorted(set(idx_prev) - set(h.nodes()))) #node removed, i.e. they belong to the (k-1)-core but not to the k-core
    idx_prev=list(h.nodes())
    for j in idx_removed:
        k_shell_dict[j]=kk-k_step
    D = len(idx_prev)
    if D==0:
        break #stop iterative procedure when the k-core is empty

pk.dump(k_shell_dict, open('K-shell_dict_{0}_proj_k.pck'.format(dataset),'wb')) #k-coreness